# [Building a Spam Filter with Naive Bayes](https://app.dataquest.io/m/433/guided-project%3A-building-a-spam-filter-with-naive-bayes)

## Content
- [Background](#Background)
- [ETL](#ETL)
- [Separating the dataset](#Separating-the-dataset)

To help readers gain context into your project, use the first Markdown cell of the notebook to add a title and a short introduction where you concisely explain what the project is about and what your goal is in this project (the title and the introduction are tentative at this point, so don't spend too much time here — you can come back at the end of your work to refine them).

Open the SMSSpamCollection file using the read_csv() function from the pandas package.

The data points are tab separated, so we'll need to use the sep='\t' parameter for our read_csv() function.
The dataset doesn't have a header row, which means we need to use the header=None parameter, otherwise the first row will be wrongly used as the header row.
Use the names=['Label', 'SMS'] parameter to name the columns as Label and SMS.
Explore the dataset a little.

Find how many rows and columns it has.
Find what percentage of the messages is spam and what percentage is ham ("ham" means non-spam).

In [1]:
# Importing packages for data management 
import pandas as pd    # Importing pandas
import numpy as np     # Importing numpy
import datetime as dt  # Importing datetime
import re              # Importing regular expression
import warnings        # To suppress warning alert
warnings.filterwarnings('ignore')
#Change setting to avoid dataframe from truncating
pd.options.display.max_rows = 500
pd.options.display.width = 500
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 500
# Displaying all output from code cell. Default value = 'last_expr'.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
sms = pd.read_csv("SMSSpamCollection", sep='\t', header=None, names=['Label','SMS'])

In [3]:
print("\nSMSSpamCollection dataset has {} rows and {} columns".format(sms.shape[0],sms.shape[1]))
sms['Label'].value_counts(normalize=True)*100


SMSSpamCollection dataset has 5572 rows and 2 columns


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

In [4]:
sms.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


Start by randomizing the entire dataset by using the DataFrame.sample() method.

Use the frac=1 parameter to randomize the entire dataset.
Use the random_state=1 parameter to make sure your results are reproducible.
Split the randomized dataset into a training and a test set. The training set should account for 80% of the dataset, and the remaining 20% of the data should be the test set.

Find the percentage of spam and ham in both the training and the test set. Are the percentages similar to what we have in the full dataset?

## Separating the dataset 

In [5]:
# Randomizing the dataset 
learning = sms.sample(frac=0.8,random_state=1)
test = sms.sample(frac=0.2,random_state=1)
# Checking shape and label proportion for learning dataset
print("\nLearning dataset has {} rows.".format(learning.shape[0]))
learning['Label'].value_counts(normalize=True)*100
# Checking shape and label proportion for test dataset
print("Test dataset has {} rows.".format(test.shape[0]))
test['Label'].value_counts(normalize=True)*100


Learning dataset has 4458 rows.


ham     86.54105
spam    13.45895
Name: Label, dtype: float64

Test dataset has 1114 rows.


ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Remove all the punctuation from the SMS column. You can use the regex '\W' to detect any character that is not from a-z, A-Z or 0-9.

For instance, the function re.sub('\W', ' ', 'Secret!! Money, goods.' ) strips the punctuation marks and outputs the string 'Secret Money goods '.

For simplicity, you can use the Series.str.replace() method.
For each message, transform every letter in every word to lower case. You may want to use the Series.str.lower() method.

In [6]:
# Changing to lower case and replacing all punctuation 
def clean_punctuation(x):
    result = re.sub('\W', ' ', x)
    return result

# Applying function and changing case to lower
learning['SMS'] = learning['SMS'].apply(clean_punctuation).str.lower()
test['SMS'] = test['SMS'].apply(clean_punctuation).str.lower()

# Quick look at the dataframes
learning.head(3)
test.head(3)

,Label,SMS
1078,ham,yep by the pretty sculpture
4028,ham,yes princess are you going to make me moan
958,ham,welp apparently he retired


,Label,SMS
1078,ham,yep by the pretty sculpture
4028,ham,yes princess are you going to make me moan
958,ham,welp apparently he retired


Create a vocabulary for the messages in the training set. The vocabulary should be a Python list containing all the unique words across all messages, where each word is represented as a string.

Begin by transforming each message from the SMS column into a list by splitting the string at the space character — use the Series.str.split() method.
Initiate an empty list named vocabulary.

Iterate over the the SMS column (each message in this column should be a list of strings by the time you start this loop).
Using a nested loop, iterate each message in the SMS column (each message should be a list of strings) and append each string (word) to the vocabulary list.

Transform the vocabulary list into a set using the set() function. This will remove the duplicates from the vocabulary list.

Transform the vocabulary set back into a list using the list() function.

In [7]:
# Transforming messages in SMS column into list 
learning['SMS'] = learning['SMS'].str.split()
test['SMS'] = test['SMS'].str.split()

In [9]:
# Creating vocabulary from the SMS column
vocabulary = []

for row in learning['SMS']:
    for n in row:
        if n not in vocabulary:
            vocabulary.append(n)
        else:
            continue

The for loop function in the above should prevent any duplicate in the vocabulary list.
Quick check to confirm that there's no duplicate:

In [10]:
len(vocabulary) == len(set(vocabulary))
len(vocabulary)

True

7783

Once we're sure that there's no duplicate we can move on to the next step.

Run the code you see above to get the word_counts_per_sms dictionary. In case you want to do a bit of exploration, note that this is a large dictionary, and printing it all is not recommended (you should rather use a for loop and print only the first five or so key-value pairs).

Transform word_counts_per_sms into a DataFrame using pd.DataFrame().
Concatenate the DataFrame we just built above with the DataFrame containing the training set (this way, we'll also have the Label and the SMS columns). Use the pd.concat() function.

In [11]:
word_counts_per_sms = {unique_word: [0] * len(learning['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(learning['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [8]:
# Resetting the index to allow the dataframes to be concatenated correctly
learning = learning.reset_index(drop=True)

In [12]:
training_df = pd.DataFrame(word_counts_per_sms)
training_set = pd.concat([learning,training_df],axis=1)
training_set

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,you,going,to,make,me,moan,welp,apparently,he,retired,havent,i,forgot,2,ask,ü,all,smth,there,s,a,card,on,da,present,lei,how,want,write,or,sign,it,ok,thk,got,then,u,wan,come,now,wat,kfc,its,tuesday,only,buy,meals,no,gravy,mark,dear,was,sleeping,p,pa,nothing,problem,ill,be,lt,gt,my,uncles,in,atlanta,wish,guys,great,semester,phone,which,your,another,number,greatest,test,of,courage,earth,is,bear,defeat,without,losing,heart,gn,tc,dai,what,this,can,send,resume,id,am,late,will,at,freemsg,why,haven,t,replied,text,m,randy,sexy,female,and,live,local,luv,hear,from,netcollex,ltd,08700621170150p,per,msg,reply,stop,end,k,when,re,way,congrats,mobile,3g,videophones,r,yours,call,09061744553,videochat,wid,ur,mates,play,java,games,dload,polyh,music,noline,rentl,bx420,ip4,5we,150pm,please,leave,topic,sorry,for,telling,that,ooooooh,tell,get,yoville,hi,yijue,meet,11,tmr,show,world,about,europe,well,must,pain,catch,know,mean,texting,bill,3,33,65,so,thats,not,bad,yeah,where,class,pin,fighting,with,easy,either,win,lose,bt,fightng,some1,who,close,dificult,if,still,dude,up,teresa,hope,have,been,okay,didnt,these,people,called,them,they,had,received,package,since,dec,just,thot,ld,like,do,fantastic,year,best,reading,plus,really,bam,first,aid,usmle,work,done,time,coming,...,astronomer,starer,election,recount,hitler,eleven,twelve,perpetual,dd,onam,sirji,tata,aig,recieve,channel,teletext,kay,gauti,sehwag,odi,lodging,uworld,qbank,assessment,mittelschmertz,paracetamol,ikea,spelled,caps,among,mess,bullshit,hanks,lotsly,09058091870,m26,3uz,weirdo,woo,hoo,hppnss,sorrow,goodfriend,stubborn,sucker,hospitals,suckers,fans,0870141701216,120p,bbq,6ish,09058094455,jurong,amore,09096102316,impatient,belligerent,splash,dessert,warner,83118,colin,farrell,swat,kiosk,mre,achan,amma,taught,becaus,verifying,prabu,meg,sections,clearer,1million,ppt150x3,box403,w1t1jy,0870737910216yrs,tiring,concentrating,throwing,performed,plumbers,wrench,borrow,arestaurant,squid,dosomething,wall,08712402779,ibm,hp,yalru,lyfu,astne,innu,mundhe,lyf,ali,halla,ke,bilo,edhae,ovr,vargu,meow,velachery,natalie,165,natalie2k9,nordstrom,qi,suddenly,holy,weren,95,pax,deposit,sq825,arrival,interest,figuring,favorite,oyster,sashimi,rumbling,outbid,simonwatson5120,shinco,plyr,smsrewards,notifications,ndship,needle,4few,conected,checkup,aka,pap,smear,attention,spreadsheet,determine,entire,mirror,apeshit,swimming,jacuzzi,dismissial,formally,accidant,tookplace,ghodbandar,slovely,excited,7cfca1a,07090201529,helens,princes,garments,elections,restock,scrounge,rhythm,establish,attraction,named,sorrows,craziest,proove,praises,curry,makiing,sambar,07815296484,41782,lingo,69969,bcmsfwc1n3xx,free2day,george,89080,0870241182716,sometme,andrews,costing,offering,09066368470,shb,chapel,frontierville,progress,non,cantdo,anythingtomorrow,myparents,aretaking,outfor,katexxx,08700469649,box420,virgins,4fil,sexual,theirs,69911,leanne,cutting,ooh,4got,moseley,weds,bruv,09053750005,310303,08718725756,140ppm,dan,reminded,cme,hos,occupied,armenia,swann,abbey,09066660100,2309,harry,potter,phoenix,readers,inconsiderate,nag,recession,hence,genes,prakesh,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0